In [11]:
import pandas as pd

pd.set_option("display.max_rows", 300)
import rapidfuzz
from joblib import Parallel, delayed
from tqdm import tqdm
import re
import awswrangler as wr
import boto3
import copy
import itertools
from pprint import pprint


In [12]:
wr.config.s3_endpoint_url = "http://192.168.1.7:8333"
srcBucketName = "multilabel_df"
labelDf = wr.s3.read_parquet(
    path=f"s3://{srcBucketName}/",
    dataset=True
)
labelDf

,vision_bonnet,vision_bumper_front,vision_grille,vision_headlamp_rh,vision_headlamp_lh,vision_door_front_lh,vision_door_front_rh,vision_engine,vision_bumper_rear,vision_misc,...,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear,CaseID,Circumstances_of_Accident
0,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,12657878,Collision- Head to Rear (Insured Hit TP)
1,1,1,0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,12657888,Lost control- Overturned
2,1,1,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,12657890,
3,1,1,1,0,0,0,1,1,0,1,...,1,0,0,0,0,0,0,0,12657900,Collided into animal
4,1,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,12657903,Collision- Head to Rear (Insured Hit TP)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621787,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10147236,
621788,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10936139,Collision- Head to Rear (TP Hit Insured)
621789,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10174259,
621790,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10174498,


In [13]:
mappingDf = pd.read_csv("/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/tmp/complete_view_mapping.csv")

In [14]:
mappingDf["lvl_3_desc"] = mappingDf["lvl_3_desc"].str.strip()

In [15]:
view = ["front_view", "rear_view", "front_view_left", "front_view_right", "rear_view_left", "rear_view_right"]
viewToPart = dict()
for v in view:
    partsInView = mappingDf[mappingDf[v] == 1]["lvl_3_desc"].unique().tolist()
    viewToPart[v] = ["vision_" + x.replace(" ", "_") for x in partsInView]
pprint(viewToPart)
    

{'front_view': ['vision_bonnet',
                'vision_bumper_front',
                'vision_engine',
                'vision_front_panel',
                'vision_grille',
                'vision_headlamp_lh',
                'vision_headlamp_rh',
                'vision_misc',
                'vision_non_external',
                'vision_windscreen_front'],
 'front_view_left': ['vision_bonnet',
                     'vision_bumper_front',
                     'vision_door_front_lh',
                     'vision_door_rear_lh',
                     'vision_engine',
                     'vision_fender_front_lh',
                     'vision_front_panel',
                     'vision_headlamp_lh',
                     'vision_non_external',
                     'vision_misc',
                     'vision_wheel',
                     'vision_windscreen_front'],
 'front_view_right': ['vision_bonnet',
                      'vision_bumper_front',
                      'vision_door_front_r

In [16]:
vehicleType= "Hatchback - 5 Dr"
localFileMetadata = f"/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/imgs_metadata/{vehicleType}.parquet"

In [17]:
localFilesDf = pd.read_parquet(localFileMetadata)
localFilesDf["StdDocDesc"] = localFilesDf["StdDocDesc"].str.replace(" ", "_").str.lower()
localFilesDf.sort_values(by="CaseID", inplace=True)
localFilesDf

,CaseID,iDOCID,StdDocDesc,filename,url
298726,10000289,440220959.0,front_view_left,10000289_440220959.JPG,http://192.168.1.7:8888/buckets/raw_imgs/10000...
4104,10000289,440221764.0,front_view,10000289_440221764.JPG,http://192.168.1.7:8888/buckets/raw_imgs/10000...
920513,10000289,440220884.0,front_view_right,10000289_440220884.JPG,http://192.168.1.7:8888/buckets/raw_imgs/10000...
1763554,10000289,440221007.0,rear_view_left,10000289_440221007.JPG,http://192.168.1.7:8888/buckets/raw_imgs/10000...
2385341,10000289,440220972.0,rear_view_right,10000289_440220972.JPG,http://192.168.1.7:8888/buckets/raw_imgs/10000...
...,...,...,...,...,...
1529710,13560974,619674199.0,front_view_right,13560974_619674199.JPG,http://192.168.1.7:8888/buckets/raw_imgs/13560...
2995371,13562414,620042550.0,rear_view_right,13562414_620042550.JPG,http://192.168.1.7:8888/buckets/raw_imgs/13562...
1530564,13562414,620042555.0,front_view_right,13562414_620042555.JPG,http://192.168.1.7:8888/buckets/raw_imgs/13562...
2373583,13562414,620042551.0,rear_view_left,13562414_620042551.JPG,http://192.168.1.7:8888/buckets/raw_imgs/13562...


In [18]:
viewDfMap = dict()
for v, parts in viewToPart.items():
    tempDf = labelDf[parts + ["CaseID"]]
    tempDf["view"] = v
    viewFilesDf = localFilesDf[localFilesDf["StdDocDesc"] == v]
    viewFilesDf.drop_duplicates(subset=["CaseID"], inplace=True)
    tempDf = tempDf.merge(viewFilesDf[["filename", "CaseID"]], on="CaseID")
    viewDfMap[v] = tempDf


/tmp/ipykernel_1217241/903975900.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempDf["view"] = v
/tmp/ipykernel_1217241/903975900.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  viewFilesDf.drop_duplicates(subset=["CaseID"], inplace=True)
/tmp/ipykernel_1217241/903975900.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [19]:
for view, df in viewDfMap.items():
    imgLabelFilename = f"{vehicleType}_{view}_img_labels.csv"
    wr.s3.to_csv(
        df=df,
    path=f"s3://imgs_labels/{imgLabelFilename}",

    )

In [20]:
for view, df in viewDfMap.items():
    imgLabelFilename = f"{vehicleType}_{view}_img_labels.csv"
    df = wr.s3.read_csv(
    path=f"s3://imgs_labels/{imgLabelFilename}",

    )
    print(df)

      Unnamed: 0  vision_bonnet  vision_bumper_front  vision_engine  \
0              0              1                    1              0   
1              1              1                    0              0   
2              2              1                    1              1   
3              3              1                    0              1   
4              4              1                    1              1   
...          ...            ...                  ...            ...   
8999        8999              0                    0              0   
9000        9000              0                    0              0   
9001        9001              0                    0              1   
9002        9002              0                    0              1   
9003        9003              0                    0              1   

      vision_front_panel  vision_grille  vision_headlamp_lh  \
0                      0              0                   0   
1                    